# Endpoints em TypeScript


In [ ]:
import type { APIRoute } from 'astro';
import { encodeBase32LowerCaseNoPadding } from "@/libs/authn/util";
import { clientRepository, authorizationRequestRepository } from '@/libs';
import { sessionProvider } from '@/libs/authn';

export const GET: APIRoute = async ({ cookies, url, redirect }) => {
  const sessionToken = cookies.get('session')?.value;

  if (!sessionToken) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { session } = await sessionProvider.validateSessionToken(sessionToken)

  if (!session) {
    return redirect(`/login?redirect_uri=${encodeURIComponent(url.toString())}`);
  }

  const { user_id } = session

  const requestData = Object.fromEntries(url.searchParams.entries())

  const { response_type, client_id } = requestData

  const client = clientRepository.get({
    id: client_id
  })

  if (!client) {
    return new Response(null, { status: 400 })
  }

  const { callback_endpoint } = client

  switch (response_type) {
    case "code": {
      const { code_challenge, code_challenge_method } = requestData

      if (code_challenge_method.toLowerCase() !== "s256") {
        return new Response(null, { status: 400 })
      }

      const code = generateAuthorizationCode()

      authorizationRequestRepository.create({
        code,
        client_id,
        project_id: client.project_id,
        code_challenge,
        user_id
      });

      return redirect(`${callback_endpoint}?code=${code}`);
    }
    default: {
      return new Response(null, { status: 400 })
    }
  }
}

function generateAuthorizationCode() {
  const bytes = new Uint8Array(32);
  crypto.getRandomValues(bytes);

  return encodeBase32LowerCaseNoPadding(bytes)
}
